In [1]:
%load_ext autoreload
%autoreload 2

import os,sys,warnings
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
np.set_printoptions(legacy='1.25')
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 8
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu
yu.flag_fast=False

enss=['b']
ens2full={'a24':'cA211.53.24','a':'cA2.09.48','b':'B64','c':'C80','d':'D96'}
ens2label={'a24':'A24','a':'A48','b':'B64','c':'C80','d':'D96'}
ens2a={'a24':0.0908,'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692} # fm
ens2N={'a24':24,'a':48,'b':64,'c':80,'d':96}
ens2N_T={'a24':24*2,'a':48*2,'b':64*2,'c':80*2,'d':96*2}

# ens2mN={'a':931}
# ens2mpiC={'a':131}
# ens2mpi0={'a':111}

ens2amu={'a24':0.0053,'a':0.0009,'b':0.00072}
ens2ZP={'a24':(0.4670,0.0046),'a':(0.500,0.030)}

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

ens='b'

In [2]:
path='/capstor/store/cscs/userlab/s1174/lyan/code/projectData2/01_Nsgm/cB211.072.64_base/data_from_others/thrp-conn.h5'
path2='/capstor/store/cscs/userlab/s1174/lyan/code/projectData2/01_Nsgm/cB211.072.64_base/data_from_others/NST_b.h5_cB64'
with h5py.File(path) as f, h5py.File(path2) as f2:
    cfgs_conn=[{'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]]+cfg[:4] for cfg in f['gS/up/dt10'].keys()]
    cfgs_disc=[cfg.decode() for cfg in f2['cfgs']]
    cfgs=list(set(cfgs_conn).intersection(cfgs_disc))
    cfgs.sort()
    cfgs_old=[cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]] for cfg in cfgs]
    print(len(cfgs_conn),len(cfgs_disc),len(cfgs))
    dic={}
    for i,cfg in enumerate(cfgs_disc):
        dic[cfg]=i
    cfgs_disc_map=[dic[cfg] for cfg in cfgs]
    
    tfs_conn=[int(dt[2:]) for dt in f['gS/up'].keys()]
    tfs_conn.sort()
    tfs_disc=list(set([int(fla.split('_deltat_')[-1]) for fla in f2['diags/N-j/data'].keys()]))
    tfs_disc.sort()
    tfs=list(set(tfs_conn).intersection(tfs_disc))
    tfs.sort()
    assert(len(tfs_conn)==len(tfs))
    
    c2pt=np.real(yu.jackknife([f[f'twop/{cfg}'] for cfg in cfgs_old])) 
    c2pt=None # not clear what it is, avoid using
    c3ptDic_conn={tf:np.real(yu.jackknife([f[f'gS/up/dt{tf}/{cfg}'][:] + f[f'gS/dn/dt{tf}/{cfg}'][:] for cfg in cfgs_old])) for tf in tfs}
    c2ptCorrDic={tf:np.real(yu.jackknife([f[f'dt{tf}_twop/{cfg}'][()] for cfg in cfgs_old])) for tf in tfs}
    
    # print(f2.keys())
    print(f2['diags/N/data/N2_N2'])
    print(f2['diags/N/opabs'][[0,1]])
    print(f2['diags/N-j/data'].keys())
    print(f2['diags/N-j/opabs'][[0,5]])
    print(f2['VEV/j/data/j+'])
    # print(f2[f'diags/N-j/data/N2_j+_N2_deltat_10'])
    
    c3ptDic_disc_nosub={tf:np.real(yu.jackknife(\
        (f2[f'diags/N-j/data/N2_j+_N2_deltat_{tf}'][cfgs_disc_map,:,0,0]+\
         f2[f'diags/N-j/data/N2_j+_N2_deltat_{tf}'][cfgs_disc_map,:,5,0])/2\
             )) for tf in tfs_disc}
    
    c2pt_disc=np.real(yu.jackknife((f2['diags/N/data/N2_N2'][:,:,0]+f2['diags/N/data/N2_N2'][:,:,1])/2))
    vev=np.real(yu.jackknife(f2['VEV/j/data/j+'][cfgs_disc_map,0]))
    c3ptDic_disc={tf:c3ptDic_disc_nosub[tf]-c2pt_disc[:,tf:tf+1]*vev[:,None] for tf in tfs_disc}
    c3ptDic={tf:c3ptDic_conn[tf]+c3ptDic_disc[tf] for tf in tfs}
    
    symmetrizeQ=False
    if symmetrizeQ:
        func=lambda datDic:{tf:(dat[:,:]+dat[:,::-1])/2 for tf,dat in datDic.items()}
        c3ptDic_conn=func(c3ptDic_conn)
        c3ptDic_disc=func(c3ptDic_disc)
        c3ptDic=func(c3ptDic)

749 743 743
<HDF5 dataset "N2_N2": shape (743, 36, 4), type "<c8">
[b'g;0,0,0;G1g;a;l1;N_g;0,0,0;G1g;a;l1;N'
 b'g;0,0,0;G1g;a;l2;N_g;0,0,0;G1g;a;l2;N']
<KeysViewHDF5 ['N2_j+_N2_deltat_10', 'N2_j+_N2_deltat_11', 'N2_j+_N2_deltat_12', 'N2_j+_N2_deltat_13', 'N2_j+_N2_deltat_14', 'N2_j+_N2_deltat_15', 'N2_j+_N2_deltat_16', 'N2_j+_N2_deltat_17', 'N2_j+_N2_deltat_18', 'N2_j+_N2_deltat_19', 'N2_j+_N2_deltat_20', 'N2_j+_N2_deltat_21', 'N2_j+_N2_deltat_22', 'N2_j+_N2_deltat_23', 'N2_j+_N2_deltat_24', 'N2_j+_N2_deltat_25', 'N2_j+_N2_deltat_26', 'N2_j+_N2_deltat_27', 'N2_j+_N2_deltat_28', 'N2_j+_N2_deltat_29', 'N2_j+_N2_deltat_3', 'N2_j+_N2_deltat_30', 'N2_j+_N2_deltat_4', 'N2_j+_N2_deltat_5', 'N2_j+_N2_deltat_6', 'N2_j+_N2_deltat_7', 'N2_j+_N2_deltat_8', 'N2_j+_N2_deltat_9', 'N2_j-_N2_deltat_10', 'N2_j-_N2_deltat_11', 'N2_j-_N2_deltat_12', 'N2_j-_N2_deltat_13', 'N2_j-_N2_deltat_14', 'N2_j-_N2_deltat_15', 'N2_j-_N2_deltat_16', 'N2_j-_N2_deltat_17', 'N2_j-_N2_deltat_18', 'N2_j-_N2_deltat_19', 'N2_

In [3]:
settings={
    'fitmins_1st':range(8,25+1),
    'fitmins_2st':range(1,12+1),
    'fitmins_3st':range(1,6+1),
    'ylim1':[0.6,1.7],
    'ylim1':[0.88,1.08],
    'ylim2':[0.9,1.04],
    'ylim3':[0.9,3.0],
    'xlim1':[0,2.7],
    'xlim2':[0,2.2],
    'xlim3':[0,1.1],
}

chi2Size=9
xUnit=ens2a[ens]; yUnit=ens2aInv[ens]/1000

func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))
func_mEff_1st=lambda t,E0: np.log(func_C2pt_1st(t,E0,1)/func_C2pt_1st(t+1,E0,1))
func_mEff_2st=lambda t,E0,dE1,rc1: np.log(func_C2pt_2st(t,E0,1,dE1,rc1)/func_C2pt_2st(t+1,E0,1,dE1,rc1))
func_mEff_3st=lambda t,E0,dE1,rc1,dE2,rc2: np.log(func_C2pt_3st(t,E0,1,dE1,rc1,dE2,rc2)/func_C2pt_3st(t+1,E0,1,dE1,rc1,dE2,rc2))

def run(corrQ=False,meffQ=True,pdf=None):
    fig, axd = plt.subplot_mosaic([['f1','f1','f1'],['f2','f2','f3']],figsize=(24,10))
    (ax1,ax2,ax3)=(axd[key] for key in ['f1','f2','f3'])
    if meffQ:
        fig.suptitle('Correlated fit to meff' if corrQ else 'Uncorrelated fit to meff',fontsize=44)
    else:
        fig.suptitle('Correlated fit to C2pt' if corrQ else 'Uncorrelated fit to C2pt',fontsize=44)
    
    ax1.set_xlabel(r'$t$ [fm]')
    ax2.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax3.set_xlabel(r'$t_{\mathrm{min}}$ [fm]')
    ax1.set_ylabel(r'$m_N^{\mathrm{eff}}$ [GeV]')
    ax2.set_ylabel(r'$m_N$ [GeV]')
    ax3.set_ylabel(r'$E_1$ [GeV]')
    ax1.set_ylim(settings['ylim1'])
    ax2.set_ylim(settings['ylim2'])
    ax3.set_ylim(settings['ylim3'])
    ax1.set_xlim(settings['xlim1'])
    ax2.set_xlim(settings['xlim2'])
    ax3.set_xlim(settings['xlim3'])
    
    mN_exp=0.938; mp_exp,mn_exp=(0.93827,0.93957)
    ax1.axhline(y=mN_exp,color='black',linestyle = '--', marker='')
    ax2.axhline(y=mN_exp,color='black',linestyle = '--', marker='', label=r'$m_N^{\mathrm{exp}}=$'+'%0.3f'%mN_exp)
    C2pt_jk=c2pt_disc
    C2pt_mean,C2pt_err=yu.jackme(C2pt_jk)
    C2pt_rela=np.abs(C2pt_err/C2pt_mean)
    func=lambda C2pt: np.log(C2pt/np.roll(C2pt,-1,axis=0))
    mEff_jk=yu.jackmap(func,C2pt_jk)
    (mEff_mean,mEff_err)=yu.jackme(mEff_jk)
    mEff_rela=np.abs(mEff_err/mEff_mean)
    temp=[(i,rela) for i,rela in enumerate(mEff_rela if meffQ else C2pt_rela) if rela>0.2 and i!=0]
    fitmax=temp[0][0]-1
    # print(fitmax)
    
    tmin=1; tmax=fitmax+1
    plt_x=np.arange(tmin,tmax)*xUnit; plt_y=mEff_mean[tmin:tmax]*yUnit; plt_err=mEff_err[tmin:tmax]*yUnit
    ax1.errorbar(plt_x,plt_y,plt_err,color='black',fmt='s')

    pars0_initial=[0.4,0.5,2,0.8,1] if meffQ else [0.4,1e-8,0.5,2,0.8,1]
    DNpar=1 if meffQ else 0
    
    fits_all=[]
    # 1st fits
    color='r'
    fitmins=settings['fitmins_1st']
    pars0=pars0_initial[:2-DNpar]
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            if meffQ:
                return func_mEff_1st(tList,*pars)
            return func_C2pt_1st(tList,*pars)
        y_jk=mEff_jk[:,tList] if meffQ else C2pt_jk[:,tList]
        pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,y_jk,pars0,mask=None if corrQ else 'uncorrelated')
        pars0=np.mean(pars_jk,axis=0)
        fits.append([fitmin,pars_jk,chi2_jk,Ndof])
        fits_all.append([('1st',fitmin),pars_jk[:,:1],chi2_jk,Ndof])
        
    pars_jk,props_jk=yu.jackMA(fits)
    ind_mpf=np.argmax(np.mean(props_jk,axis=0))    
    pars_mean,pars_err=yu.jackme(pars_jk)
    pars0=pars_mean
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=pars_mean[0]*yUnit; plt_err=pars_err[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2,label=r'$m_N^{\mathrm{1st}}=$'+yu.un2str(plt_y,plt_err))
    for i,fit in enumerate(fits):
        fitmin,pars_jk,chi2_jk,Ndof=fit
        (pars_mean,pars_err)=yu.jackme(pars_jk)
        chi2R=np.mean(chi2_jk)/Ndof
        plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_err=pars_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white' if i==ind_mpf else None)
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
    
    # 2st fits
    color='g'
    fitmins=settings['fitmins_2st']
    pars0=np.hstack([pars0,pars0_initial[2-DNpar:4-DNpar]])
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            if meffQ:
                return func_mEff_2st(tList,*pars)
            return func_C2pt_2st(tList,*pars)
        y_jk=mEff_jk[:,tList] if meffQ else C2pt_jk[:,tList]
        pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,y_jk,pars0,mask=None if corrQ else 'uncorrelated')
        pars0=np.mean(pars_jk,axis=0)
        fits.append([fitmin,pars_jk,chi2_jk,Ndof])
        fits_all.append([('2st',fitmin),pars_jk[:,:1],chi2_jk,Ndof])
    pars_jk,props_jk=yu.jackMA(fits)
    ind_mpf=np.argmax(np.mean(props_jk,axis=0))    
    pars_jk[:,1]=pars_jk[:,0]+pars_jk[:,2-DNpar]
    pars_mean,pars_err=yu.jackme(pars_jk)
    pars0=pars_mean
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=pars_mean[0]*yUnit; plt_err=pars_err[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$m_N^{\mathrm{2st}}=$'+yu.un2str(plt_y,plt_err))
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=pars_mean[1]*yUnit; plt_err=pars_err[1]*yUnit
    ax3.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2)      
    for i,fit in enumerate(fits):
        fitmin,pars_jk,chi2_jk,Ndof=fit
        pars_jk[:,1]=pars_jk[:,0]+pars_jk[:,2-DNpar]
        (pars_mean,pars_err)=yu.jackme(pars_jk)
        chi2R=np.mean(chi2_jk)/Ndof
        
        plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_err=pars_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white' if i==ind_mpf else None)
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
        plt_x=fitmin*xUnit; plt_y=pars_mean[1]*yUnit; plt_err=pars_err[1]*yUnit
        ax3.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white' if i==ind_mpf else None)
        ylim=ax3.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax3.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
    
    # 3st fits
    color='b'
    fitmins=settings['fitmins_3st']
    pars0=np.hstack([pars0,pars0_initial[4-DNpar:6-DNpar]])
    fits=[]
    for fitmin in fitmins:
        tList=np.arange(fitmin,fitmax)
        def fitfunc(pars):
            if meffQ:
                return func_mEff_3st(tList,*pars)
            return func_C2pt_3st(tList,*pars)
        y_jk=mEff_jk[:,tList] if meffQ else C2pt_jk[:,tList]
        pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,y_jk,pars0,mask=None if corrQ else 'uncorrelated')
        pars0=np.mean(pars_jk,axis=0)
        fits.append([fitmin,pars_jk,chi2_jk,Ndof])
        fits_all.append([('3st',fitmin),pars_jk[:,:1],chi2_jk,Ndof])
    pars_jk,props_jk=yu.jackMA(fits)
    ind_mpf=np.argmax(np.mean(props_jk,axis=0))    
    pars_jk[:,1]=pars_jk[:,0]+pars_jk[:,2-DNpar]
    pars_mean,pars_err=yu.jackme(pars_jk)
    pars0=pars_mean
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=pars_mean[0]*yUnit; plt_err=pars_err[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$m_N^{\mathrm{3st}}=$'+yu.un2str(plt_y,plt_err))
    plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xUnit; plt_y=pars_mean[1]*yUnit; plt_err=pars_err[1]*yUnit
    ax3.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2)      
    for i,fit in enumerate(fits):
        fitmin,pars_jk,chi2_jk,Ndof=fit
        pars_jk[:,1]=pars_jk[:,0]+pars_jk[:,2-DNpar]
        (pars_mean,pars_err)=yu.jackme(pars_jk)
        chi2R=np.mean(chi2_jk)/Ndof
        
        plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_err=pars_err[0]*yUnit
        ax2.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white' if i==ind_mpf else None)
        ylim=ax2.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax2.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
        
        plt_x=fitmin*xUnit; plt_y=pars_mean[1]*yUnit; plt_err=pars_err[1]*yUnit
        ax3.errorbar(plt_x,plt_y,plt_err,fmt='s',color=color,mfc='white' if i==ind_mpf else None)
        ylim=ax3.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        ax3.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center') 
    
    color='orange'
    pars_jk,props_jk=yu.jackMA(fits_all)
    ind_mpf=np.argmax(np.mean(props_jk,axis=0))
    pars_mean,pars_err=yu.jackme(pars_jk)
    pars0=pars_mean
    plt_x=settings['xlim2']; plt_y=pars_mean[0]*yUnit; plt_err=pars_err[0]*yUnit
    ax2.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$m_N^{\mathrm{nst}}=$'+yu.un2str(plt_y,plt_err) + f'; MPF: {fits_all[ind_mpf][0][0]}')    
    
    ax2.legend(loc=(0.6,0.5),fontsize=12)
    
    plt.tight_layout()
    if pdf!=None:
        pdf.savefig(fig)
    else:
        if meffQ:
            plt.savefig('fig/fit_meff_cor.pdf' if corrQ else 'fig/fit_meff_unc.pdf')
        else:
            plt.savefig('fig/fit_2pt_cor.pdf' if corrQ else 'fig/fit_2pt_unc.pdf')
    plt.close()


import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("fig/fit2pt.pdf")
run(False,True,pdf=pdf)
run(True,False,pdf=pdf)
run(True,True,pdf=pdf)
run(False,False,pdf=pdf)
pdf.close()

/capstor/store/cscs/userlab/s1174/lyan/code/glwc2/project2/01_Nsgm/util.py:101: RuntimeWarning: Number of calls to function has reached maxfev = 1000.
  pars=leastsq(fitfunc_wrapper2,pars_mean)[0]
/capstor/store/cscs/userlab/s1174/lyan/code/glwc2/project2/01_Nsgm/util.py:101: RuntimeWarning: Number of calls to function has reached maxfev = 1400.
  pars=leastsq(fitfunc_wrapper2,pars_mean)[0]


In [5]:
# check charge paper PhysRevD.111.054505 using amu and a from arXiv:2411.08852
t_gS=np.array([13.4,1.9])
t_ZP=0.4638 #(61)
t_a_iso=0.07948 #(11)fm
t_a_charge=0.07957 #(13)fm
t_amu_iso=0.0006669 #(28)
t_amu_charge=0.00072
t_sgm=np.array([50.1,7.2])

t_aInv_iso=1/(t_a_iso*hbarc)
t_aInv_charge=1/(t_a_charge*hbarc)
print(t_sgm,'quote')
print(t_gS/t_ZP*t_amu_iso*t_aInv_charge,'amu=iso,a=charge')
print(t_gS/t_ZP*t_amu_charge*t_aInv_iso,'amu=charge,a=iso')
print(t_gS/t_ZP*t_amu_iso*t_aInv_iso,'both iso')
print(t_gS/t_ZP*t_amu_charge*t_aInv_charge,'both charge')

[50.1  7.2] quote
[47.77629841  6.77425127] amu=iso,a=charge
[51.63875671  7.32191326] amu=charge,a=iso
[47.8303984   6.78192216] both iso
[51.58034916  7.3136316 ] both charge


In [ ]:
settings={
    # 'tmins2pt':range(8,9+1),
    # 'tfmins':[8,10],
    # 'tcmins':[1,2],
    # 'tmins2pt':[4,5,6,7,8,9,10],
    # 'tfmins':[8,10,12,14,16,18],
    # 'tcmins':[1,2,3,4,5,6,7,8,9],
    'tmins2pt':[5,6,7,8,9],
    'tfmins':[8,10,12],
    'tcmins':[1,2,3],
    
    # 'tmins2pt':[8],
    # 'tfmins':[10],
    # 'tcmins':[2],
    'ylim':[0,80],
}

colorDic={8:'pink',10:'orange',12:'lightgreen',14:'purple',16:'skyblue',18:'brown',20:'blue'}
fmtDic={4:'v',5:'d',6:'o',7:'s',8:'<',9:'>',10:'^'}

func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))
func_mEff_1st=lambda t,E0: np.log(func_C2pt_1st(t,E0,1)/func_C2pt_1st(t+1,E0,1))
func_mEff_2st=lambda t,E0,dE1,rc1: np.log(func_C2pt_2st(t,E0,1,dE1,rc1)/func_C2pt_2st(t+1,E0,1,dE1,rc1))
func_mEff_3st=lambda t,E0,dE1,rc1,dE2,rc2: np.log(func_C2pt_3st(t,E0,1,dE1,rc1,dE2,rc2)/func_C2pt_3st(t+1,E0,1,dE1,rc1,dE2,rc2))

func_C3pt_2st=lambda tf,tc,E0a,E0b,a00,dE1a,dE1b,ra01,ra10,ra11: a00*np.exp(-E0a*(tf-tc))*np.exp(-E0b*tc)*(1 + ra01*np.exp(-dE1b*tc) + ra10*np.exp(-dE1a*(tf-tc)) + ra11*np.exp(-dE1a*(tf-tc))*np.exp(-dE1b*tc)) \
    if a00!=0 else np.exp(-E0a*(tf-tc))*np.exp(-E0b*tc)*(ra01*np.exp(-dE1b*tc) + ra10*np.exp(-dE1a*(tf-tc)) + ra11*np.exp(-dE1a*(tf-tc))*np.exp(-dE1b*tc))
func_ratio_2st=lambda tf,tc,g,dE1,rc1,ra01,ra11:func_C3pt_2st(tf,tc,1,1,g,dE1,dE1,ra01,ra01,ra11)/func_C2pt_2st(tf,1,1,dE1,rc1)

def run(pdf=None):
    meffQ=False; corrQ=True; discQ=False
    
    if discQ:    
        tfs=list(c3ptDic_disc.keys())
        tfs.sort()
        tfs=[tf for tf in tfs if tf<=24]
        settings['ylim']=[-5,20]
    else:
        tfs=list(c3ptDic.keys())
        tfs.sort()    
    
    c2pt_jk=c2pt_disc
    # ratio_jk={tf:c3ptDic[tf][:,:]/c2pt_disc[:,tf:tf+1] for tf in tfs}
    # ratio_jk={tf:c3ptDic[tf][:,:]/c2ptCorrDic[tf][:,None] for tf in tfs}
    ratio_jk={tf:c3ptDic_conn[tf][:,:]/c2ptCorrDic[tf][:,None]+c3ptDic_disc[tf][:,:]/c2pt_disc[:,tf:tf+1] for tf in tfs}
    if discQ:
        ratio_jk={tf:c3ptDic_disc[tf][:,:]/c2pt_disc[:,tf:tf+1] for tf in tfs}
    

    
    C2pt_mean,C2pt_err=yu.jackme(c2pt_jk)
    C2pt_rela=np.abs(C2pt_err/C2pt_mean)
    func=lambda C2pt: np.log(C2pt/np.roll(C2pt,-1,axis=0))
    mEff_jk=yu.jackmap(func,c2pt_jk)
    (mEff_mean,mEff_err)=yu.jackme(mEff_jk)
    mEff_rela=np.abs(mEff_err/mEff_mean)
    temp=[(i,rela) for i,rela in enumerate(mEff_rela if meffQ else C2pt_rela) if rela>0.2 and i!=0]
    tmax2pt=temp[0][0]-1

    pars0=[0.4,1e-8,10,0.5,1,1,1]
    if discQ:
        pars0=[0.4,1e-8,1,0.5,1,1,1]
    fits=[]
    for i1,tmin2pt in enumerate(settings['tmins2pt']):
        for i2,tfmin in enumerate(settings['tfmins']):
            for i3,tcmin in enumerate(settings['tcmins']):
                print(f"({i1+1}/{len(settings['tmins2pt'])},{i2+1}/{len(settings['tfmins'])},{i3+1}/{len(settings['tcmins'])})",end='               \r')
                if tfmin+1-tcmin <= tcmin:
                    continue    

                tfs_fit=[tf for tf in tfs if tf>=tfmin]
                tcs_fit={tf:np.arange(tcmin,tf//2+1) for tf in tfs_fit}
                tfs2pt_fit=np.arange(tmin2pt,tmax2pt+1)
                
                c2pt_fit=c2pt_jk[:,tfs2pt_fit]
                ratio_fit=np.concatenate([ratio_jk[tf][:,tcs_fit[tf]]for tf in tfs_fit],axis=1)
                
                c_fit=np.concatenate([c2pt_fit,ratio_fit],axis=1)
                
                def fitfunc(pars):
                    E0,c0,g,dE1,rc1,ra01,ra11=pars
                    t2pt=func_C2pt_2st(tfs2pt_fit,E0,c0,dE1,rc1)
                    t3pt=[func_ratio_2st(tf,tcs_fit[tf],g,dE1,rc1,ra01,ra11) for tf in tfs_fit]
                    t=[t2pt]+t3pt
                    return np.concatenate(t)
                pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,c_fit,pars0,mask=None if corrQ else 'uncorrelated')
                pars0=np.mean(pars_jk,axis=0)
                fits.append([(tmin2pt,tfmin,tcmin),pars_jk,chi2_jk,Ndof])
                
    fig, axs = yu.getFigAxs(1,2,sharey=True)
    xunit=ens2a[ens]; yunit=[1,0.4638,ens2amu[ens]*ens2aInv[ens],][2]
    ax=axs[0,0]
    ax.set_ylim(settings['ylim'])
    for tf in tfs:
        t_cut=1
        ratio=ratio_jk[tf]
        mean,err=yu.jackme(ratio)
        if tf%2==0:
            plt_x=np.arange(t_cut - tf//2,tf//2 - (t_cut-1))*xunit; plt_y=mean[t_cut:tf+1-t_cut]*yunit; plt_yerr=err[t_cut:tf+1-t_cut]*yunit
        else:
            plt_x=(np.arange(t_cut - tf//2,tf//2 - (t_cut-1)+1)-0.5)*xunit; plt_y=mean[t_cut:tf+1-t_cut]*yunit; plt_yerr=err[t_cut:tf+1-t_cut]*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color=colorDic[tf] if tf in colorDic else None,mfc='white')  
        
    ax=axs[0,1]
    color='red'
    pars_jk,props_jk=yu.jackMA(fits)
    ind_mpf=np.argmax(np.mean(props_jk,axis=0))
    print(fits[ind_mpf][0],np.mean(fits[ind_mpf][2]),fits[ind_mpf][3],np.mean(fits[ind_mpf][2])/fits[ind_mpf][3])
    pars_mean,pars_err=yu.jackme(pars_jk)
    pars0=pars_mean
    plt_x=np.array([settings['tcmins'][0]-0.5,settings['tcmins'][-1]+0.5])*xunit; plt_y=pars_mean[2]*yunit; plt_err=pars_err[2]*yunit
    ax.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$\sigma_{\pi N}=$'+yu.un2str(plt_y,plt_err)+f'; MPF:{fits[ind_mpf][0]}')
    for i,fit in enumerate(fits):
        fitlabel,pars_jk,chi2_jk,Ndof=fit
        (tmin2pt,tfmin,tcmin)=fitlabel
        (pars_mean,pars_err)=yu.jackme(pars_jk)
        chi2R=np.mean(chi2_jk)/Ndof
        
        plt_x=tcmin*xunit; plt_y=pars_mean[2]*yunit; plt_err=pars_err[2]*yunit
        ax.errorbar(plt_x,plt_y,plt_err,fmt=fmtDic[tmin2pt],color=colorDic[tfmin] if tfmin in colorDic else None,mfc='white' if i==ind_mpf else None)
        # ylim=ax.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        # ax.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')
    
    ax.legend()
    
    plt.tight_layout()
    if pdf!=None:
        pdf.savefig(fig)
    else:
        plt.savefig('fig/fit3pt.pdf')
    plt.close()

run()

(8, 8, 2) 72.6106726607787 62 1.171139881625463


In [22]:
# reproduce charge paper

settings={
    # 'tmins2pt':range(8,9+1),
    # 'tfmins':[8,10],
    # 'tcmins':[1,2],
    
    # 'tmins2pt':[7,8,9],
    # 'tfmins':[8,10,12,14,16,18],
    # 'tcmins':[1,2,3,4,5,6,7,8,9],
    
    # same with charge paper
    'tmins2pt':[5,6,7,8,9],
    'tfmins':[8,10,12],
    'tcmins':[1,2,3],

    # for test
    # 'tmins2pt':[8],
    # 'tfmins':[10],
    # 'tcmins':[2],
    'ylim':[0,80]
}

colorDic={8:'pink',10:'orange',12:'lightgreen',14:'purple',16:'skyblue',18:'brown',20:'blue'}
fmtDic={4:'v',5:'d',6:'o',7:'s',8:'<',9:'>',10:'^'}

func_C2pt_1st=lambda t,E0,c0: c0*np.exp(-E0*t)
func_C2pt_2st=lambda t,E0,c0,dE1,rc1: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t))
func_C2pt_3st=lambda t,E0,c0,dE1,rc1,dE2,rc2: c0*np.exp(-E0*t)*(1 + rc1*np.exp(-dE1*t) + rc2*np.exp(-dE2*t))
func_mEff_1st=lambda t,E0: np.log(func_C2pt_1st(t,E0,1)/func_C2pt_1st(t+1,E0,1))
func_mEff_2st=lambda t,E0,dE1,rc1: np.log(func_C2pt_2st(t,E0,1,dE1,rc1)/func_C2pt_2st(t+1,E0,1,dE1,rc1))
func_mEff_3st=lambda t,E0,dE1,rc1,dE2,rc2: np.log(func_C2pt_3st(t,E0,1,dE1,rc1,dE2,rc2)/func_C2pt_3st(t+1,E0,1,dE1,rc1,dE2,rc2))

func_C3pt_2st=lambda tf,tc,E0a,E0b,a00,dE1a,dE1b,ra01,ra10,ra11: a00*np.exp(-E0a*(tf-tc))*np.exp(-E0b*tc)*(1 + ra01*np.exp(-dE1b*tc) + ra10*np.exp(-dE1a*(tf-tc)) + ra11*np.exp(-dE1a*(tf-tc))*np.exp(-dE1b*tc)) \
    if a00!=0 else np.exp(-E0a*(tf-tc))*np.exp(-E0b*tc)*(ra01*np.exp(-dE1b*tc) + ra10*np.exp(-dE1a*(tf-tc)) + ra11*np.exp(-dE1a*(tf-tc))*np.exp(-dE1b*tc))
func_ratio_2st=lambda tf,tc,g,dE1,rc1,ra01,ra11:func_C3pt_2st(tf,tc,1,1,g,dE1,dE1,ra01,ra01,ra11)/func_C2pt_2st(tf,1,1,dE1,rc1)

def run(pdf=None):
    meffQ=False; corrQ=False; halfRainbowQ=False

    tfs=list(c3ptDic.keys())
    tfs.sort()
    tfs_disc=list(c3ptDic_disc.keys())
    tfs_disc.sort()
    tfs_disc=[tf for tf in tfs_disc if tf<=24]
    
    c2pt_jk=c2pt_disc
    ratio_jk_conn={tf:c3ptDic_conn[tf][:,:]/c2ptCorrDic[tf][:,None] for tf in tfs}
    ratio_jk_disc={tf:c3ptDic_disc[tf][:,:]/c2pt_disc[:,tf:tf+1] for tf in tfs_disc}
    # ratio_jk={tf:c3ptDic[tf][:,:]/c2pt_disc[:,tf:tf+1] for tf in tfs}
    # ratio_jk={tf:c3ptDic[tf][:,:]/c2ptCorrDic[tf][:,None] for tf in tfs}
    ratio_jk={tf:c3ptDic_conn[tf][:,:]/c2ptCorrDic[tf][:,None]+c3ptDic_disc[tf][:,:]/c2pt_disc[:,tf:tf+1] for tf in tfs}
    
    C2pt_mean,C2pt_err=yu.jackme(c2pt_jk)
    C2pt_rela=np.abs(C2pt_err/C2pt_mean)
    func=lambda C2pt: np.log(C2pt/np.roll(C2pt,-1,axis=0))
    mEff_jk=yu.jackmap(func,c2pt_jk)
    (mEff_mean,mEff_err)=yu.jackme(mEff_jk)
    mEff_rela=np.abs(mEff_err/mEff_mean)
    temp=[(i,rela) for i,rela in enumerate(mEff_rela if meffQ else C2pt_rela) if rela>0.2 and i!=0]
    tmax2pt=temp[0][0]-1

    pars0=[0.4,1e-8,8,0.5,1,1,1]
    fits=[]
    for i1,tmin2pt in enumerate(settings['tmins2pt']):
        for i2,tfmin in enumerate(settings['tfmins']):
            for i3,tcmin in enumerate(settings['tcmins']):
                print(f"({i1+1}/{len(settings['tmins2pt'])},{i2+1}/{len(settings['tfmins'])},{i3+1}/{len(settings['tcmins'])})",end='               \r')
                if tfmin+1-tcmin <= tcmin:
                    continue    

                tfs_fit=[tf for tf in tfs if tf>=tfmin]
                if halfRainbowQ:
                    tcs_fit={tf:np.arange(tcmin,tf//2+1) for tf in tfs_fit}
                else:
                    tcs_fit={tf:np.arange(tcmin,tf+1-tcmin) for tf in tfs_fit}
                tfs2pt_fit=np.arange(tmin2pt,tmax2pt+1)
                
                c2pt_fit=c2pt_jk[:,tfs2pt_fit]
                ratio_fit=np.concatenate([ratio_jk_conn[tf][:,tcs_fit[tf]]for tf in tfs_fit],axis=1)
                
                c_fit=np.concatenate([c2pt_fit,ratio_fit],axis=1)
                
                def fitfunc(pars):
                    E0,c0,g,dE1,rc1,ra01,ra11=pars
                    t2pt=func_C2pt_2st(tfs2pt_fit,E0,c0,dE1,rc1)
                    t3pt=[func_ratio_2st(tf,tcs_fit[tf],g,dE1,rc1,ra01,ra11) for tf in tfs_fit]
                    t=[t2pt]+t3pt
                    return np.concatenate(t)
                pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,c_fit,pars0,mask=None if corrQ else 'uncorrelated')
                pars0=np.mean(pars_jk,axis=0)
                fits.append([(tmin2pt,tfmin,tcmin),pars_jk,chi2_jk,Ndof])        
    fits_conn=fits
    
    # pars0=[0.4,1e-8,1,0.5,1,1,1]
    pars0[2]=1
    fits=[]
    for i1,tmin2pt in enumerate(settings['tmins2pt']):
        for i2,tfmin in enumerate(settings['tfmins']):
            for i3,tcmin in enumerate(settings['tcmins']):
                print(f"({i1+1}/{len(settings['tmins2pt'])},{i2+1}/{len(settings['tfmins'])},{i3+1}/{len(settings['tcmins'])})",end='               \r')
                if tfmin+1-tcmin <= tcmin:
                    continue    

                tfs_fit=[tf for tf in tfs_disc if tf>=tfmin]
                if halfRainbowQ:
                    tcs_fit={tf:np.arange(tcmin,tf//2+1) for tf in tfs_fit}
                else:
                    tcs_fit={tf:np.arange(tcmin,tf+1-tcmin) for tf in tfs_fit}
                tfs2pt_fit=np.arange(tmin2pt,tmax2pt+1)
                
                c2pt_fit=c2pt_jk[:,tfs2pt_fit]
                ratio_fit=np.concatenate([ratio_jk_disc[tf][:,tcs_fit[tf]]for tf in tfs_fit],axis=1)
                
                c_fit=np.concatenate([c2pt_fit,ratio_fit],axis=1)
                
                def fitfunc(pars):
                    E0,c0,g,dE1,rc1,ra01,ra11=pars
                    t2pt=func_C2pt_2st(tfs2pt_fit,E0,c0,dE1,rc1)
                    t3pt=[func_ratio_2st(tf,tcs_fit[tf],g,dE1,rc1,ra01,ra11) for tf in tfs_fit]
                    t=[t2pt]+t3pt
                    return np.concatenate(t)
                pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,c_fit,pars0,mask=None if corrQ else 'uncorrelated')
                pars0=np.mean(pars_jk,axis=0)
                fits.append([(tmin2pt,tfmin,tcmin),pars_jk,chi2_jk,Ndof])        
    fits_disc=fits
    
    pars0=[0.4,1e-8,10,0.5,1,1,1]
    fits=[]
    for i1,tmin2pt in enumerate(settings['tmins2pt']):
        for i2,tfmin in enumerate(settings['tfmins']):
            for i3,tcmin in enumerate(settings['tcmins']):
                print(f"({i1+1}/{len(settings['tmins2pt'])},{i2+1}/{len(settings['tfmins'])},{i3+1}/{len(settings['tcmins'])})",end='               \r')
                if tfmin+1-tcmin <= tcmin:
                    continue    

                tfs_fit=[tf for tf in tfs if tf>=tfmin]
                if halfRainbowQ:
                    tcs_fit={tf:np.arange(tcmin,tf//2+1) for tf in tfs_fit}
                else:
                    tcs_fit={tf:np.arange(tcmin,tf+1-tcmin) for tf in tfs_fit}
                tfs2pt_fit=np.arange(tmin2pt,tmax2pt+1)
                
                c2pt_fit=c2pt_jk[:,tfs2pt_fit]
                ratio_fit=np.concatenate([ratio_jk[tf][:,tcs_fit[tf]]for tf in tfs_fit],axis=1)
                
                c_fit=np.concatenate([c2pt_fit,ratio_fit],axis=1)
                
                def fitfunc(pars):
                    E0,c0,g,dE1,rc1,ra01,ra11=pars
                    t2pt=func_C2pt_2st(tfs2pt_fit,E0,c0,dE1,rc1)
                    t3pt=[func_ratio_2st(tf,tcs_fit[tf],g,dE1,rc1,ra01,ra11) for tf in tfs_fit]
                    t=[t2pt]+t3pt
                    return np.concatenate(t)
                pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,c_fit,pars0,mask=None if corrQ else 'uncorrelated')
                pars0=np.mean(pars_jk,axis=0)
                fits.append([(tmin2pt,tfmin,tcmin),pars_jk,chi2_jk,Ndof])        
    fits_full=fits
    
    print()
    fig, axs = yu.getFigAxs(3,2,sharex='col',sharey='row')
    yu.addRowHeader(axs,['conn','disc','c+d'])
    xunit=ens2a[ens]; yunit=[1,0.4638,ens2amu[ens]*ens2aInv[ens],][2]
    for irow in range(3):
        ax=axs[irow,0]
        if irow!=1:
            ax.set_ylim(settings['ylim'])
        else:
            ax.set_ylim([-5,20])
        tmp=tfs if irow!=1 else tfs_disc
        for tf in tmp:
            t_cut=1
            ratio=[ratio_jk_conn,ratio_jk_disc,ratio_jk][irow][tf]
            mean,err=yu.jackme(ratio)
            if tf%2==0:
                plt_x=np.arange(t_cut - tf//2,tf//2 - (t_cut-1))*xunit; plt_y=mean[t_cut:tf+1-t_cut]*yunit; plt_yerr=err[t_cut:tf+1-t_cut]*yunit
                ax.errorbar(plt_x,plt_y,plt_yerr,color=colorDic[tf] if tf in colorDic else None,mfc='white')  
            else:
                plt_x=(np.arange(t_cut - tf//2,tf//2 - (t_cut-1)+1)-0.5)*xunit; plt_y=mean[t_cut:tf+1-t_cut]*yunit; plt_yerr=err[t_cut:tf+1-t_cut]*yunit
                ax.errorbar(plt_x,plt_y,plt_yerr,color=colorDic[tf] if tf in colorDic else None,mfc='white')
            
        ax=axs[irow,1]
        color='red'
        fits=[fits_conn,fits_disc,fits_full][irow]
        pars_jk,props_jk=yu.jackMA(fits)
        ind_mpf=np.argmax(np.mean(props_jk,axis=0))
        print(fits[ind_mpf][0],np.mean(fits[ind_mpf][2]),fits[ind_mpf][3],np.mean(fits[ind_mpf][2])/fits[ind_mpf][3])
        pars_mean,pars_err=yu.jackme(pars_jk)
        pars0=pars_mean
        plt_x=np.array([settings['tcmins'][0]-0.5,settings['tcmins'][-1]+0.5])*xunit; plt_y=pars_mean[2]*yunit; plt_err=pars_err[2]*yunit
        ax.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color=color,alpha=0.2, label=r'$\sigma_{\pi N}=$'+yu.un2str(plt_y,plt_err)+f'; MPF:{fits[ind_mpf][0]}')
        for i,fit in enumerate(fits):
            fitlabel,pars_jk,chi2_jk,Ndof=fit
            (tmin2pt,tfmin,tcmin)=fitlabel
            (pars_mean,pars_err)=yu.jackme(pars_jk)
            chi2R=np.mean(chi2_jk)/Ndof
            
            plt_x=tcmin*xunit; plt_y=pars_mean[2]*yunit; plt_err=pars_err[2]*yunit
            ax.errorbar(plt_x,plt_y,plt_err,fmt=fmtDic[tmin2pt],color=colorDic[tfmin] if tfmin in colorDic else None,mfc='white' if i==ind_mpf else None)
            # ylim=ax.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
            # ax.annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=color,size=chi2Size,ha='center')

        if irow==2:
            pars_jk_conn,props_jk=yu.jackMA(fits_conn)
            pars_jk_disc,props_jk=yu.jackMA(fits_disc)
            pars_jk=pars_jk_conn+pars_jk_disc
            pars_mean,pars_err=yu.jackme(pars_jk)
            plt_x=np.array([settings['tcmins'][0],settings['tcmins'][-1]+1])*xunit; plt_y=pars_mean[2]*yunit; plt_err=pars_err[2]*yunit
            ax.fill_between(plt_x,plt_y-plt_err,plt_y+plt_err,color='grey',alpha=0.2, label=r'$\sigma_{\pi N}=$'+yu.un2str(plt_y,plt_err))
            print(plt_y,plt_err,r'$\sigma_{\pi N}=$'+yu.un2str(plt_y,plt_err))
            pars_mean_conn,pars_err_conn=yu.jackme(pars_jk_conn)
            pars_mean_disc,pars_err_disc=yu.jackme(pars_jk_disc)
            plt_y=(pars_mean_conn[2]+pars_mean_disc[2])*yunit; plt_err=np.sqrt(pars_err_conn[2]**2+pars_err_disc[2]**2)*yunit
            print(plt_y,plt_err,r'$\sigma_{\pi N}=$'+yu.un2str(plt_y,plt_err))
        
        ax.legend()
    
    plt.tight_layout()
    if pdf!=None:
        pdf.savefig(fig)
    else:
        plt.savefig('fig/fit3ptCDF_chargePaper.pdf')
    plt.close()

run()

(5/5,3/3,3/3)               
(7, 10, 1) 52.13315518849996 105 0.4965062398904758
(9, 8, 1) 133.56306389061294 274 0.48745643755698154
(9, 8, 1) 60.722309003215344 110 0.5520209909383214
53.822703100472445 6.7404791271411755 $\sigma_{\pi N}=$53.8(6.7)
53.82270310047236 7.019754822816877 $\sigma_{\pi N}=$53.8(7.0)
